# SMuRF basic checks

## Changes
[Describe new version changes and why they were implemented]

## Start server
Navigate to the release directory and start the server with the `run.py` script.

## Setup system

### Imports

In [ ]:
import matplotlib
matplotlib.use('Agg')
%matplotlib inline
import matplotlib.pyplot as plt
import ipywidgets as widgets
import os
import numpy as np

import pysmurf.client

### Get a SmurfControl instance

In [ ]:
slot = 4
epics_prefix = f"smurf_server_s{slot}"
config_file = os.path.abspath("/usr/local/src/pysmurf/cfg_files/rflab/experiment_rflab_2xLB_backplane.cfg")
shelf_man = "shm-smrf-sp01"

In [ ]:
S = pysmurf.client.SmurfControl(
    epics_root=epics_prefix,
    cfg_file=config_file,
    setup=False,
    make_logfile=False,
    shelf_manager=shelf_man
)

What versions of everything are we running?

In [ ]:
pysmurf.__version__

In [ ]:
S._caget(f'{epics_prefix}:AMCc:RogueVersion', as_string=True)

In [ ]:
S._caget(f'{epics_prefix}:AMCc:SmurfApplication:SmurfVersion', as_string=True)

In [ ]:
S._caget(f'{epics_prefix}:AMCc:SmurfApplication:StartupArguments', as_string=True)

In [ ]:
S.get_fpga_git_hash_short()

### Run setup
Check that the output contains no errors and the function returns `True`

In [ ]:
assert S.setup()

### Check JESD is locked
(JESD204 is a standard interface for communication links to the FPGA.)

In [ ]:
for i in range(2):
    print(f"JESD in bay {i}:", S.check_jesd(i))

## Loopback tests
For these tests, no cold resonators are required, and the system should be configured in loopback.

In [ ]:
# convenience function for displaying the output of a given function in tabs per band
def do_tabs(func, *args, bands=S.bands, **kwargs):
    # create a tab for each band
    tab_outputs = [widgets.Output() for b in bands]
    for i, band in enumerate(bands):
        with tab_outputs[i]:
            func(band, *args, **kwargs)

    # display as tabs
    tab = widgets.Tab(children=tab_outputs)
    for i, band in enumerate(bands):
        tab.set_title(i, f"Band {band}")
    display(tab)

### Estimate phase delay
[describe what this is doing and how to interpret results]

**What is this doing when there are no resonators connected?** It's calling `full_band_resp`, which itself calls `find_freq` to measure the band response. Then it fits for a linear trend in the phase. I'm not sure I understand how this works with/without resonators in the system...

In [ ]:
do_tabs(S.estimate_phase_delay)

### Full band response
[describe what this is doing and what to look for]

In [ ]:
do_tabs(S.full_band_resp, make_plot=True, show_plot=True, bands=S.bands[:2])

In [ ]:
freq_dsp, resp_dsp = S.find_freq(0, make_plot=True, show_plot=True)

In [ ]:
freq_dsp[0]

# TODO:
- currently tests transceivers and low-level DSP
- not analog low-freq low-noise stuff (not in loopback)
- data streaming: track on the cable delay or fixed tones:
    - stream for some amount time in loopback
    - may be more of a hardware test - out of scope
- make a list of existing tests and what they do

### Resonator tests

**TODO:** Need to adapt these once there is a cold system to test with

Enable drains and check drain currents

In [ ]:
S.C.write_ps_en(11)
S.get_amplifier_biases()

In [ ]:
#adjust gate until hemt_Id is 4mA.  50K is B&Z amp that doesn't have a gate.
S.set_hemt_gate_voltage(0.265)

In [ ]:
S.get_amplifier_biases()

Gate voltage corresponding to 4mA is similar to what it usually is.

In [ ]:
S.hemt_bit_to_V

Set and get UC and DC attenuators.  Matching UC=12, DC=12 to keep ADC from saturating during `find_freqs`.

In [ ]:
band=0

S.set_att_uc(band,24) # took out 6 dB attenuator to run with phase shifter, but that's no longer inline so adding 6dB UC att
S.set_att_dc(band,12)

S.get_att_uc(band,write_log=True)
S.get_att_dc(band,write_log=True)

**I guess this is also not doing anything useful without resonators??**

Take a full band response.

In [ ]:
S.bands

In [ ]:
band = 3
S.full_band_resp(band,make_plot=True,show_plot=True)

In [ ]:
freqs, resps = S.find_freq(band, tone_power=12,grad_cut=0.01, amp_cut=0.1,make_plot=True,show_plot=True)

Looks like...exactly the same as when I ran this on 12/2/22!

In [ ]:
freqs, resps = S.find_freq(band, start_freq=10, stop_freq=150, tone_power=12,grad_cut=0.01, amp_cut=0.1,make_plot=True,show_plot=True)

Still need to figure out why (0,0) is showing up in this plot?

In [ ]:
# Tune
S.setup_notches(band,new_master_assignment=True)

# Drop DC for optimal S/N, and to match settings used with jackhammer.
S.set_att_dc(band,2)

# Refine tuning
S.run_serial_gradient_descent(band)

# Add fixed tone to monitor RF phase
S.set_fixed_tone(4250,12)

# Refine eta calibrations
S.run_serial_eta_scan(band)
S.plot_tune_summary(band,eta_scan=True,show_plot=True)

The crash here is a known "feature" of pysmurf v7.2.0.

Turn off bad channels.  480 is collided.  We'll just turn off that one for now, but need to fix this bug so we can see others.

In [ ]:
for chan in [480]:
    S.channel_off(band,chan)

Turn off feedback for fixed tone and make sure it's center frequency is zero.

In [ ]:
S.set_feedback_enable_channel(0,0,0)
print(S.get_center_frequency_mhz_channel(0,0))

Set AC flux ramp mode

In [ ]:
S.set_cryo_card_relay_bit(16,0)
bin(S.get_cryo_card_relays())

After running `setup_notches`, we're tracking on all 65 resonances - let's make sure neither the ADC or DAC are saturated;

In [ ]:
S.check_dac_saturation(0)
S.check_adc_saturation(0)

Not clipping either the RF DAC or ADC.  Tracking setup

In [ ]:
 S.tracking_setup(band, channel = S.which_on(band), reset_rate_khz=4, make_plot=True, save_plot=True, show_plot=True, 
                  lms_freq_hz=None, n_phi0=5, meas_flux_ramp_amp=True, fraction_full_scale=0.2,feedback_gain=2048,lms_gain=0,
                  return_data=False, feedback_start_frac=0.02, feedback_end_frac=0.98,nsamp = 2**18)

All tracking curves look good.  Histogram of flux ramp amplitude looks nearly identical to on 12/22.  Similar percent full scale for 5 phi0 ; here it's

In [ ]:
S.get_fraction_full_scale()

And on 12/22 it was 22.940%.

Take noise data - first a short 5 min acq.

In [ ]:
S.take_noise_psd(300,show_plot=True)

Those epics stalls are a little concerning, although it's possible other people are doing stuff on the SMuRF server right now.  I'm not crazy about the compression in some of the flux ramp frames, that wasn't as noticible on 12/2/22.

In [ ]:
#Import library
from IPython.display import Image
# Load image from local storage
plt.ion()

Image(filename = "/data/smurf_data/20230215/1676482954/plots/1676484915_noise_timestream_b0_ch340.png", width = 600, height = 300)

Also compute noise the same way it's done in sodetlib.

In [ ]:
t,d,m=S.read_stream_data('/data/smurf_data/20230215/1676482954/outputs/1676484915.dat')

In [ ]:
def get_wls_sodetlib(t,d, nperseg=2**16, fmin=10., fmax=20., pA_per_phi0=9e6):
    """
    Gets white-noise levels for each channel from the axis manager returned
    by smurf_ops.load_session.
    Args
    ----
    t : np.array
        times returned by S.read_stream_data function
    d : np.array
        tod returned by S.read_stream_data function
    nperseg : int
        nperseg to be passed to welch
    fmin : float
        Min frequency to use for white noise mask
    fmax : float
        Max freq to use for white noise mask
    pA_per_phi0 : float
        S.pA_per_phi0 unit conversion. This will eventually make its way
        into the axis manager, but for now I'm just hardcoding this here
        as a keyword argument until we get there.
    Returns
    --------
    wls : array of floats
        Array of the white-noise level for each channel, indexed by readout-
        channel number
    band_medians : array of floats
        Array of the median white noise level for each band.
    """
    fsamp = 1./np.median(np.diff(t*1.e-9))
    fs, pxx = signal.welch(d * pA_per_phi0 / (2*np.pi),
                           fs=fsamp, nperseg=nperseg)
    pxx = np.sqrt(pxx)
    fmask = (fmin < fs) & (fs < fmax)
    wls = np.median(pxx[:, fmask], axis=1)
    return wls

In [ ]:
np.median(get_wls_sodetlib(t,d,fmin=0.1,fmax=1.))

In [ ]:
print(np.median(get_wls_sodetlib(t,d)))

In [ ]:
band=0
bm={}
bm[band]=np.median(get_wls_sodetlib(t,d))
wls=get_wls_sodetlib(t,d)
plt.figure()
plt.hist(wls[wls < 200], bins = 20,alpha = 0.5,label = 'Noise per Channel')
print(bm[band])
plt.axvline(bm[band],ls = ':',color = 'C1',lw = 3,label = 
                f'Band Median:\n{np.round(bm[band],2)}'+' pA/$\sqrt{Hz}$')
plt.xlabel('NEI [pA/$\sqrt{Hz}$]',fontsize = 16)
plt.ylabel('Counts',fontsize = 16)
plt.legend(fontsize = 16)
fig = plt.gcf()
fig.patch.set_facecolor('white')
plt.show()

In [ ]:
#Import library
from IPython.display import Image
# Load image from local storage
plt.ion()

Image(filename = "/data/smurf_data/20230215/1676482954/plots/1676484915_noise_timestream_b0_ch340.png", width = 600, height = 300)

In [ ]:
#Import library
from IPython.display import Image
# Load image from local storage
plt.ion()

Image(filename = "/data/smurf_data/20230215/1676482954/plots/1676484915_noise_timestream_b0_ch000.png", width = 600, height = 300)